<a href="https://colab.research.google.com/github/RonanKMcGovern/qlora-functions/blob/loss-mask/QLoRA_for_function_calling_loss_mask_WORKING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *About*

Forked from: https://colab.research.google.com/drive/1FxlUb_H6Xirhkx4RszAgHeb2uDW7oKIH by https://www.youtube.com/@vrsen

Authored by: RonanKMcGovern of https://research.trelis.com and https://huggingface.co/Trelis

# Install

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
cache_dir = "/content/drive/My Drive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists


In [3]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.1 MB/s eta 0:00:00


In [4]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [7]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True, cache_dir=cache_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Run Notes

Llama-2-7b-chat-hf (unsharded)
Running with High RAM and T4 GPU.

CPU is at 4.8 GB
GPU is at 6.2 GB

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

In [127]:
print("EOS token:", tokenizer.eos_token)
print("EOS token id:", tokenizer.eos_token_id)

EOS token: </s>
EOS token id: 2


In [10]:
print("Pad token: ", tokenizer.pad_token)
print("Pad token ID: ", tokenizer.pad_token_id)

Using pad_token, but it is not set yet.


Pad token:  None
Pad token ID:  None


Need to set the pad token...

In [20]:
# Initialize the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Sample string
sample_string = ['hello [/INST]', 'my good friend</s>']

# Tokenize the stringified JSON object
encoded_sample = tokenizer(sample_string, truncation=True, padding=True, max_length=512, return_tensors='pt', add_special_tokens=True)

# Count the number of tokens
token_count = len(encoded_sample)

# Fetch BOS and EOS tokens
BOS_token_id = tokenizer.bos_token_id
EOS_token_id = tokenizer.eos_token_id
BOS_token = tokenizer.decode([BOS_token_id])
EOS_token = tokenizer.decode([EOS_token_id])

# Check and print BOS and EOS tokens
print(f"Beginning of the sequence: {sample_string[0]} (BOS token: {BOS_token}, id: {BOS_token_id})")
print(f"End of the sequence: {sample_string[-1]} (EOS token: {EOS_token}, id: {EOS_token_id})")

print(f"The number of tokens in the stringified JSON object is: {token_count}")

Beginning of the sequence: hello[/INST] (BOS token: <s>, id: 1)
End of the sequence: my good friend</s> (EOS token: </s>, id: 2)
The number of tokens in the stringified JSON object is: 2


In [21]:
print(encoded_sample)

{'input_ids': tensor([[    1, 22172, 29961, 29914, 25580, 29962],
        [    2,     1,   590,  1781,  5121,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1]])}


In [22]:
print(f"Model's maximum length: {tokenizer.model_max_length}")

Model's maximum length: 1000000000000000019884624838656


In [23]:
print(model.config)


LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.32.0.dev0",
  "use_cache": true,
  "vocab_size": 32000
}



## Set up LoRa

In [24]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [25]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [26]:
print_trainable_parameters(model)

trainable params: 0 || all params: 3500412928 || trainable%: 0.0


In [27]:
print(model.state_dict().keys())

odict_keys(['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.2.self_attn.k_proj.weight', 'model.layers.2.self_attn.v_proj.weight', 'model.layers.2.self_attn.o_proj.weight', 'model.layers.2.mlp.gate_proj.weight', 'mod

In [28]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [29]:
### Original LoRa config - DO NOT RUN

# from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model, config)
# print_trainable_parameters(model)

# Prepare Data

In [147]:
from datasets import load_dataset

data = load_dataset("Trelis/function_calling")

In [148]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['systemPrompt', 'userPrompt', 'assistantResponse'],
        num_rows: 11
    })
    test: Dataset({
        features: ['systemPrompt', 'userPrompt', 'assistantResponse'],
        num_rows: 5
    })
})


In [149]:
class TextDataset(Dataset):
    def __init__(self, encodings, response_lengths):
        self.encodings = encodings
        self.response_lengths = response_lengths

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}

        # Set labels to be the same as input_ids
        item["labels"] = item["input_ids"].clone()

        # Shift labels to the left and replace the last position with EOS token
        item["labels"][:-1] = item["input_ids"][1:]
        item["labels"][-1] = 2  # Replace last position with EOS token ID

        # Create a loss mask
        response_start_position = item["input_ids"].shape[0] - self.response_lengths[idx]
        item["loss_mask"] = torch.zeros_like(item["input_ids"])
        item["loss_mask"][response_start_position:] = 1

        # print("TextDataset getitem outputs:", item.keys())

        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [236]:
def prepare_dataset(dataset, tokenizer):
    # Define the roles and markers
    B_SYS, E_SYS = "<<SYS>>", "<</SYS>>"
    B_INST, E_INST = "[INST]", "[/INST]"

    # Create the formatted text with the correct roles for each part of the dialogue
    formatted_dataset = dataset.map(
        lambda x: {
            "input_text": "".join([
                f"{B_INST} {B_SYS}{x['systemPrompt'].strip()}{E_SYS} \n",
                f"{x['userPrompt'].strip()} {E_INST} \n",
                f"{x['assistantResponse'].strip()}",  # appending the EOS token in TextData...
            ]),
            "response_text": "".join([
                f"{x['assistantResponse'].strip()}",  # appending the EOS token in TextData...
            ]),
        }
    )

    # Tokenize the datasets
    encodings = tokenizer([dialogue["input_text"] for dialogue in formatted_dataset], truncation=True, padding=True, max_length=512, return_tensors='pt', add_special_tokens=True)

    # Tokenize the response one by one without padding and special tokens for the purpose of calculating length
    response_lengths = [len(tokenizer.encode(dialogue["response_text"], truncation=True, max_length=512, add_special_tokens=False)) for dialogue in formatted_dataset]

    # Create TextDataset
    text_dataset = TextDataset(encodings, response_lengths)

    return text_dataset


In [237]:
# Apply function to your datasets
train_dataset = prepare_dataset(data['train'], tokenizer)
test_dataset = prepare_dataset(data['test'], tokenizer)  # assuming you have a test set named 'data['test']'

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

### Examine the datasets

In [238]:
# Print the number of items in the dataset
print(f"Number of samples in the dataset: {len(train_dataset)}")

# Get a sample item
sample_item = train_dataset[0]  # replace 0 with the index of the sample you want to examine

# Print the dimensions of the sample item
print(f"Dimensions of input_ids: {sample_item['input_ids'].shape}")
print(f"Dimensions of attention_mask: {sample_item['attention_mask'].shape}")
print(f"Dimensions of loss_mask: {sample_item['loss_mask'].shape}")
print(f"Dimensions of labels: {sample_item['labels'].shape}")

# Print some tokens from the start and end of the sample
num_tokens_to_print = 38  # replace with the number of tokens you want to print

print("\nTokens at the start of the sample:")
print(sample_item['input_ids'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][:num_tokens_to_print].tolist()))

print("\nLabels at the start of the sample:")
print(sample_item['labels'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][:num_tokens_to_print].tolist()))

print("Attention mask at the start of the sample:")
print(sample_item['attention_mask'][:num_tokens_to_print].tolist())

print("Loss mask at the start of the sample:")
print(sample_item['loss_mask'][:num_tokens_to_print].tolist())

print("\nTokens at the end of the sample:")
print(sample_item['input_ids'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][-num_tokens_to_print:].tolist()))

print("\nLabels at the end of the sample:")
print(sample_item['labels'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][-num_tokens_to_print:].tolist()))

print("Attention mask at the end of the sample:")
print(sample_item['attention_mask'][-num_tokens_to_print:].tolist())

print("Loss mask at the end of the sample:")
print(sample_item['loss_mask'][-num_tokens_to_print:].tolist())


Number of samples in the dataset: 11
Dimensions of input_ids: torch.Size([462])
Dimensions of attention_mask: torch.Size([462])
Dimensions of loss_mask: torch.Size([462])
Dimensions of labels: torch.Size([462])

Tokens at the start of the sample:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 518, 25580, 29962, 3532, 14816, 29903, 6778, 3492, 526, 263, 8444, 5925, 20255, 29889, 450, 1494, 3168]
['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '<s>', '▁[', 'INST', ']', '▁<<', 'SY', 'S', '>>', 'You', '▁are', '▁a', '▁helpful', '▁research', '▁assistant', '.', '▁The', '▁following', '▁functions']

Labels at the start of the sample:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 518, 25580, 29962, 3532, 14816, 29903, 6778, 3492, 526, 263, 8444, 5925, 20255, 29889, 450, 1494, 3168, 29892]
['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', 

In [239]:
# Define a function to print some samples
def print_samples(dataset, tokenizer, num_samples=5):
    for i, sample in enumerate(dataset):
        if i >= num_samples:  # Stop after printing the desired number of samples
            break
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        labels = sample['labels']

        # Decode the input and the label
        decoded_input = tokenizer.decode(input_ids, skip_special_tokens=False)
        decoded_label = tokenizer.decode(labels, skip_special_tokens=False)

        print(f"Sample {i+1}:")
        print(f"Input: {decoded_input}")
        print(f"Label: {decoded_label}")
        print("----------------------")

# Use the function to print some samples from your train_dataset
print_samples(train_dataset, tokenizer)

Sample 1:
Input: </s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> [INST] <<SYS>>You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use 

## Generate a sample

In [154]:
import textwrap
wrapper = textwrap.TextWrapper(width=80)

In [155]:
import re  # import regular expressions module

In [156]:
def generate(index):
    system_prompt = data['test'][index]['systemPrompt']
    user_prompt = data['test'][index]['userPrompt']
    correct_answer = data['test'][index]['assistantResponse']

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    # Define the roles and their corresponding prompts
    SYSTEM_ROLE = "system"
    USER_ROLE = "user"
    ASSISTANT_ROLE = "assistant"

    # Define your prompt template with the roles
    dialog = [
        {"role": SYSTEM_ROLE, "content": system_prompt.strip()},
        {"role": USER_ROLE, "content": user_prompt.strip()},
    ]

    # Transform dialog into a format compatible with Llama2
    dialog_transformed = [
        {
            "role": dialog[1]["role"],
            "content": f"{B_INST} {B_SYS}{dialog[0]['content']}{E_SYS}{dialog[1]['content']} {E_INST}",
        }
    ]

    # Concatenate the 'content' of the messages, maintaining the role sequence
    prompt = "".join([entry['content'] for entry in dialog_transformed])

    print("Prompt:")
    print(prompt)

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=200, do_sample=True, temperature=0.01, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    print()

    # Subtract the length of input_ids from output to get only the model's response
    output_text = tokenizer.decode(output[0, len(encoding.input_ids[0]):], skip_special_tokens=False)
    output_text = re.sub('\n+', '\n', output_text)  # remove excessive newline characters

    print("Generated Assistant Response:")
    print(output_text)

    print()

    print("Correct Assistant Response:")
    print(correct_answer)

    print()

In [157]:
generate(1)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Prompt:
[INST] <<SYS>>
You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.\",\"parameters\"

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


KeyboardInterrupt: ignored

# Training

In [240]:
import torch.nn as nn

In [241]:
class CustomTrainer(transformers.Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        loss_mask = inputs.pop("loss_mask")

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        #         # Log inputs and outputs
        # print("-"*80)
        # print("Inputs for the current batch:")
        # for key, value in inputs.items():
        #     print(f"{key}: {value}")

        # print("-"*80)
        # print("Outputs for the current batch:")
        # for key, value in outputs.items():
        #     print(f"{key}: {value}")

        # Convert logits to probabilities using softmax function - unnecessary if you just want the most likely token.
        probs = nn.functional.softmax(logits, dim=-1)

        # Get the most probable tokens
        predicted_token_ids = torch.argmax(probs, dim=-1)

        # Convert token indices to words
        input_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[0])
        true_tokens = tokenizer.convert_ids_to_tokens(labels[0])

        # Compute the loss
        loss_fct = nn.CrossEntropyLoss(reduction='none')
        losses = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

        # Apply the loss mask - technically this could be done before provided prompts are within a certain range.
        masked_loss = losses * loss_mask.view(-1)

        # Aggregate the masked losses
        loss = masked_loss.sum() / loss_mask.sum()  # normalizing by the number of tokens considered

        # Define the number of tokens you want to display
        num_tokens = 460  # Change this number to your desired number of tokens

        # # Printing formatted output
        # print("-"*80)
        # print(f"Token analysis for last {num_tokens} tokens:")
        # print("{:<20}{:<20}{:<20}{:<30}{:<30}".format("Input Token", "Predicted Token", "True Token", "Raw Loss", "Masked Loss"))
        # for i in range(-num_tokens, 0, 1):
        #     print("{:<20}{:<20}{:<20}{:<30.6f}{:<30.6f}".format(input_tokens[i], predicted_tokens[i], true_tokens[i], losses.view(-1)[i], masked_loss.view(-1)[i]))

        # print("-"*80)

        return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self):
      train_dataset = self.train_dataset
      data_collator = self.data_collator

      dataloader_params = {
          "batch_size": self.args.train_batch_size,
          "collate_fn": data_collator,
          "num_workers": self.args.dataloader_num_workers,
          "pin_memory": self.args.dataloader_pin_memory,
      }

      if not isinstance(train_dataset, torch.utils.data.IterableDataset):
          dataloader_params["sampler"] = self._get_train_sampler()
          dataloader_params["drop_last"] = self.args.dataloader_drop_last

      return DataLoader(train_dataset, **dataloader_params)

In [242]:
class CustomDataCollator: # Needed if the EOS token is to be included in training.
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        # print("Batch size:", len(batch))  # Print the batch size
        # print("First item keys:", batch[0].keys())  # Print the keys of the first item in the batch

        input_ids = torch.stack([item['input_ids'] for item in batch])
        attention_mask = torch.stack([item['attention_mask'] for item in batch])
        labels = torch.stack([item['labels'] for item in batch])
        loss_mask = torch.stack([item['loss_mask'] for item in batch])

        # print("Labels in collator:", labels)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'loss_mask': loss_mask
        }

data_collator = CustomDataCollator(tokenizer)


In [244]:
trainer = CustomTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, #turn on the eval dataset for comparisons
    args=transformers.TrainingArguments(
        num_train_epochs=2,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=3,
        max_grad_norm=0.7,
        warmup_ratio=0.05,
        max_steps=20,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=data_collator,
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [245]:
trainer.train()

Step,Training Loss
1,0.007000
2,0.000400
3,0.107800
4,0.021000
5,0.286500
6,0.023800
7,0.021400
8,0.004700
9,0.009400
10,0.000400


TrainOutput(global_step=20, training_loss=0.034740017566946335, metrics={'train_runtime': 145.8439, 'train_samples_per_second': 0.411, 'train_steps_per_second': 0.137, 'total_flos': 563179169710080.0, 'train_loss': 0.034740017566946335, 'epoch': 5.45})

# Example After Fine Tuning

In [246]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=Fals

In [251]:
generate(0)

Prompt:
[INST] <<SYS>>
You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.\",\"parameters\"

# Merge Adapters and Save Model to Hub

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# model.save_pretrained("falcon-7b-instruct-ft-adapters", push_to_hub=True) # save LORA adapters
# model.push_to_hub("falcon-7b-instruct-ft-adapters")

In [ ]:
# from transformers import AutoModelForCausalLM, PretrainedConfig
# import torch

# # reload the base model (you might need pro subscription for this)
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct", device_map={"":0}, trust_remote_code=True, torch_dtype=torch.float16)

In [ ]:
# from peft import PeftModel

# # load perf model with new adapters
# model = PeftModel.from_pretrained(
#     model,
#     "vrsen/falcon-7b-instruct-ft-adapters",
# )

In [ ]:
model = model.merge_and_unload() # merge adapters with

In [ ]:
# model.push_to_hub("falcon-7b-instruct-ft")

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
# tokenizer.push_to_hub("falcon-7b-instruct-ft", use_auth_token=True)